In [60]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.effects
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [61]:
# --- Step 1: Define Configuration, Paths, and Parameters ---

# !!! IMPORTANT !!! -> Update these paths to match your system
LABEL_PATH = "/home/punith/Desktop/cHEAL 2.o/Labels.xlsx"
AUDIO_DIR = "/home/punith/Desktop/cHEAL 2.o/RespiratoryDatabase@TR"
MODEL_SAVE_PATH = "best_copd_model.keras"

# --- Parameters for feature extraction ---
N_MELS = 128
MAX_LEN = 150 # Max length of spectrogram time axis

# --- Parameters for augmentation ---
NOISE_FACTOR = 0.005
TIME_SHIFT_MAX_SEC = 0.2
PITCH_SHIFT_STEPS = 4
TIME_STRETCH_RATE = 0.8 # Speed up by 1/0.8 = 1.25x

# --- Parameters for training ---
EPOCHS = 25 # Set to your desired number of epochs
INITIAL_LEARNING_RATE = 0.001
BATCH_SIZE = 32

In [62]:

# --- Step 2: Define Helper Functions (Augmentation & Feature Extraction) ---

def add_gaussian_noise(y, noise_factor=NOISE_FACTOR):
    noise = np.random.randn(len(y))
    augmented_data = y + noise_factor * noise
    return augmented_data.astype(type(y[0]))

def time_shift(y, sr, shift_max_sec=TIME_SHIFT_MAX_SEC):
    shift_amount = int(sr * np.random.uniform(low=-shift_max_sec, high=shift_max_sec))
    return np.roll(y, shift_amount)

def extract_log_mel_spectrogram(y, sr):
    """Extracts a log-mel spectrogram and pads/truncates it to a fixed size."""
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    log_mel = librosa.power_to_db(mel_spec)
    if log_mel.shape[1] < MAX_LEN:
        pad_width = MAX_LEN - log_mel.shape[1]
        log_mel = np.pad(log_mel, ((0, 0), (0, pad_width)), mode='constant')
    else:
        log_mel = log_mel[:, :MAX_LEN]
    return log_mel

In [63]:
# --- Step 3: Load Data, Perform Patient-Aware Split, and Prepare for Training ---

print("--- Step 3: Loading and Preparing Data ---")
df = pd.read_excel(LABEL_PATH)
df_copd = df[df["Diagnosis"].isin(["COPD0", "COPD1"])].copy()
df_copd['label_encoded'] = df_copd['Diagnosis'].apply(lambda x: 0 if x == 'COPD0' else 1)
label_dict = dict(zip(df_copd["Patient ID"], df_copd["label_encoded"]))
patient_ids = list(label_dict.keys())

# Patient-aware split to prevent data leakage
train_pids, test_pids = train_test_split(patient_ids, test_size=0.25, random_state=42)
print(f"Total Patients: {len(patient_ids)}")
print(f"Training Patients: {len(train_pids)}")
print(f"Testing Patients: {len(test_pids)}")

# Process audio, augment, and create datasets
X_train, y_train = [], []
X_test, y_test = [], []

for pid in patient_ids:
    label = label_dict[pid]
    is_training_patient = pid in train_pids
    for side in ['L', 'R']:
        for i in range(1, 7):
            fname = f"{pid}_{side}{i}.wav"
            fpath = os.path.join(AUDIO_DIR, fname)
            if not os.path.exists(fpath): continue
            
            y_audio, sr = librosa.load(fpath, sr=None)
            original_spec = extract_log_mel_spectrogram(y_audio, sr)
            
            if is_training_patient:
                # Add original and 4 augmented versions for training data
                X_train.extend([
                    original_spec,
                    extract_log_mel_spectrogram(add_gaussian_noise(y_audio), sr),
                    extract_log_mel_spectrogram(time_shift(y_audio, sr), sr),
                    extract_log_mel_spectrogram(librosa.effects.pitch_shift(y=y_audio, sr=sr, n_steps=PITCH_SHIFT_STEPS), sr),
                    extract_log_mel_spectrogram(librosa.effects.time_stretch(y=y_audio, rate=1/TIME_STRETCH_RATE), sr)
                ])
                y_train.extend([label] * 5)
            else:
                # Only add original version for testing data
                X_test.append(original_spec)
                y_test.append(label)

# Convert to numpy arrays and add channel dimension for CNN
X_train = np.array(X_train)[..., np.newaxis]
y_train = np.array(y_train)
X_test = np.array(X_test)[..., np.newaxis]
y_test = np.array(y_test)

print("\n--- Final Data Shapes ---")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

--- Step 3: Loading and Preparing Data ---
Total Patients: 11
Training Patients: 8
Testing Patients: 3

--- Final Data Shapes ---
X_train shape: (480, 128, 150, 1)
y_train shape: (480,)
X_test shape: (36, 128, 150, 1)
y_test shape: (36,)


In [50]:
# --- Step 4: Calculate Class Weights to Handle Imbalance ---

print("\n--- Step 4: Handling Class Imbalance ---")
# Calculate class weights
class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights_array))

print(f"Labels in training data: {len(y_train)} total")
print(f"Count of COPD0 (label 0): {np.sum(y_train == 0)}")
print(f"Count of COPD1 (label 1): {np.sum(y_train == 1)}")
print(f"Calculated Class Weights: {class_weights}")
print("The model will now penalize mistakes on the minority class more heavily.")


--- Step 4: Handling Class Imbalance ---
Labels in training data: 480 total
Count of COPD0 (label 0): 300
Count of COPD1 (label 1): 180
Calculated Class Weights: {0: np.float64(0.8), 1: np.float64(1.3333333333333333)}
The model will now penalize mistakes on the minority class more heavily.


In [64]:
# --- Step 5: Build and Compile the CNN Model ---

print("\n--- Step 5: Building the CNN Model ---")
model = Sequential([
    tf.keras.Input(shape=(N_MELS, MAX_LEN, 1)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') # Sigmoid for binary classification
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


--- Step 5: Building the CNN Model ---


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 128, 150, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 128, 150, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 64, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 64, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 64, 75, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 32, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 32, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 32, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 32, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 16, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 16, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │     4,718,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,812,417 (18.36 MB)

 Trainable params: 4,811,969 (18.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [26]:
# --- Step 6: Define Callbacks and Train the Model ---

# Define ALL callbacks for robust training
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_accuracy', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

print(f"\n--- Step 6: Starting Model Training for {EPOCHS} Epochs ---")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, model_checkpoint, reduce_lr],
    class_weight=class_weights # <-- Applying the calculated weights here
)


--- Step 6: Starting Model Training for 25 Epochs ---


Epoch 1/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - accuracy: 0.4514 - loss: 9.6973
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to best_copd_model.keras
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 777ms/step - accuracy: 0.4530 - loss: 9.4566 - val_accuracy: 0.3333 - val_loss: 8.3556 - learning_rate: 0.0010
Epoch 2/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.6059 - loss: 1.1197
Epoch 2: val_accuracy did not improve from 0.33333
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 742ms/step - accuracy: 0.6058 - loss: 1.1052 - val_accuracy: 0.3333 - val_loss: 13.9499 - learning_rate: 0.0010
Epoch 3/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 744ms/step - accuracy: 0.6931 - loss: 0.6321
Epoch 3: val_accuracy did not improve from 0.33333
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 758ms/step - accuracy: 0.6905 - loss: 0.6330 - val_accuracy: 0.3333 - val_loss: 8.1000 - learning_rate: 0.0010
Epoch 4/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 752ms/step - accuracy: 0.6243 - loss: 0.6112
Epoch 4: val_accuracy did not improv

In [65]:
# --- Step 7: Evaluate the Final Model ---

print("\n--- Step 7: Evaluating Best Saved Model ---")
# The best model is already loaded thanks to `restore_best_weights=True` in EarlyStopping
# Or we can explicitly load the one saved by ModelCheckpoint for certainty
print(f"Loading best model from {MODEL_SAVE_PATH} for final evaluation.")
model.load_weights(MODEL_SAVE_PATH)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")
print(f"Final Test Loss: {loss:.4f}")


--- Step 7: Evaluating Best Saved Model ---
Loading best model from best_copd_model.keras for final evaluation.

Final Test Accuracy: 61.11%
Final Test Loss: 1.0006


In [71]:
# --- Step 8: Analyze Patient Scores with the Trained Model ---

def run_patient_analysis(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    """
    Analyzes all patients with a specific diagnosis, computes the average model
    prediction score for their audio files, and returns a sorted DataFrame.
    """
    print(f"\n--- 🩺 Starting Analysis for '{target_diagnosis}' Patients ---")
    
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    
    if not patient_ids:
        print(f"No patients with diagnosis '{target_diagnosis}' found.")
        return None
        
    print(f"Found {len(patient_ids)} patients with '{target_diagnosis}' diagnosis. Analyzing files...")
    
    patient_predictions = {}
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fname = f"{pid}_{side}{i}.wav"
                fpath = os.path.join(audio_dir, fname)
                
                if not os.path.exists(fpath):
                    continue

                try:
                    y_audio, sr = librosa.load(fpath, sr=None)
                    log_mel_spec = extract_log_mel_spectrogram(y_audio, sr)
                    
                    model_input = np.expand_dims(log_mel_spec, axis=(0, -1)) # Combined expand_dims
                    
                    probability = trained_model.predict(model_input, verbose=0)[0][0]
                    scores.append(probability)
                except Exception as e:
                    print(f"Warning: Could not process file {fname}: {e}")
        # print(scores)
        if scores:
            patient_predictions[pid] = scores

    results_list = []
    for pid, scores in patient_predictions.items():
        avg_score = np.mean(scores)
        results_list.append({
            'Patient ID': pid,
            'Avg_Score': avg_score,
            'Audio_Files_Found': len(scores)
        })

    if not results_list:
        print("Could not generate any predictions.")
        return None

    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values(by='Avg_Score', ascending=sort_ascending)
    return results_df.reset_index(drop=True)


# --- Run the analysis for both groups ---

# 1. Analyze COPD1 patients to see their confidence scores (high is good)
# We sort ascending to see the patients the model is LEAST confident about first.
copd1_confidence_df = run_patient_analysis(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD1',
    sort_ascending=True
)

if copd1_confidence_df is not None:
    print("\n--- ✅ COPD1 Patient Confidence Score Results ---")
    print("'Avg_Score' is the model's confidence in the 'COPD1' prediction (closer to 1.0 is more confident).")
    print("Patients with LOW scores might be outliers or less severe cases.")
    print(copd1_confidence_df)

# 2. Analyze COPD0 patients to see their risk scores (low is good)
# We sort descending to see the "healthy" patients the model thinks are MOST at risk.
copd0_risk_df = run_patient_analysis(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD0',
    sort_ascending=False
)

if copd0_risk_df is not None:
    print("\n--- ⚠️ COPD0 Patient Risk Assessment Results ---")
    print("'Avg_Score' is the predicted risk (similarity to COPD1). Higher scores indicate higher risk.")
    print("High-scoring patients may warrant closer clinical observation.")
    print(copd0_risk_df)


--- 🩺 Starting Analysis for 'COPD1' Patients ---
Found 5 patients with 'COPD1' diagnosis. Analyzing files...

--- ✅ COPD1 Patient Confidence Score Results ---
'Avg_Score' is the model's confidence in the 'COPD1' prediction (closer to 1.0 is more confident).
Patients with LOW scores might be outliers or less severe cases.
  Patient ID  Avg_Score  Audio_Files_Found
0       H045   0.274511                 12
1       H039   0.716988                 12
2       H017   0.868227                 12
3       H043   0.899407                 12
4       H029   0.966138                 12

--- 🩺 Starting Analysis for 'COPD0' Patients ---
Found 6 patients with 'COPD0' diagnosis. Analyzing files...

--- ⚠️ COPD0 Patient Risk Assessment Results ---
'Avg_Score' is the predicted risk (similarity to COPD1). Higher scores indicate higher risk.
High-scoring patients may warrant closer clinical observation.
  Patient ID  Avg_Score  Audio_Files_Found
0       H041   0.402531                 12
1       H050   0

Showing all 12 recording's prediction value and then averaging

In [74]:
# # --- Step 8: Analyze Patient Scores with the Trained Model ---

# def run_patient_analysis(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
#     """
#     Analyzes all patients with a specific diagnosis, computes the average model
#     prediction score for their audio files, and returns a sorted DataFrame.
#     """
#     print(f"\n--- 🩺 Starting Analysis for '{target_diagnosis}' Patients ---")
    
#     df_labels = pd.read_excel(labels_path)
#     patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    
#     if not patient_ids:
#         print(f"No patients with diagnosis '{target_diagnosis}' found.")
#         return None
        
#     print(f"Found {len(patient_ids)} patients with '{target_diagnosis}' diagnosis. Analyzing files...")
    
#     patient_predictions = {}
#     for pid in patient_ids:
#         scores = []
#         for side in ['L', 'R']:
#             for i in range(1, 7):
#                 fname = f"{pid}_{side}{i}.wav"
#                 fpath = os.path.join(audio_dir, fname)
                
#                 if not os.path.exists(fpath):
#                     continue

#                 try:
#                     y_audio, sr = librosa.load(fpath, sr=None)
#                     log_mel_spec = extract_log_mel_spectrogram(y_audio, sr)
                    
#                     model_input = np.expand_dims(log_mel_spec, axis=(0, -1)) # Combined expand_dims
                    
#                     probability = trained_model.predict(model_input, verbose=0)[0][0]
#                     scores.append(probability)
#                 except Exception as e:
#                     print(f"Warning: Could not process file {fname}: {e}")
        
#         # --- MODIFICATION START ---
#         # If scores were collected for the patient, print them in the desired format
#         # before adding them to the dictionary.
#         if scores:
#             # Create a list of scores formatted to 4 decimal places as strings
#             formatted_scores = [f'{s:.4f}' for s in scores]
#             print(f"  -> Scores for Patient {pid}: {formatted_scores}")
            
#             # Store the original float scores for calculation
#             patient_predictions[pid] = scores
#         # --- MODIFICATION END ---

#     results_list = []
#     for pid, scores in patient_predictions.items():
#         avg_score = np.mean(scores)
#         results_list.append({
#             'Patient ID': pid,
#             'Avg_Score': avg_score,
#             'Audio_Files_Found': len(scores)
#         })

#     if not results_list:
#         print("Could not generate any predictions.")
#         return None

#     results_df = pd.DataFrame(results_list)
#     results_df = results_df.sort_values(by='Avg_Score', ascending=sort_ascending)
#     return results_df.reset_index(drop=True)


# # --- Run the analysis for both groups ---

# # 1. Analyze COPD1 patients to see their confidence scores (high is good)
# # We sort ascending to see the patients the model is LEAST confident about first.
# copd1_confidence_df = run_patient_analysis(
#     trained_model=model,
#     labels_path=LABEL_PATH,
#     audio_dir=AUDIO_DIR,
#     target_diagnosis='COPD1',
#     sort_ascending=True
# )

# if copd1_confidence_df is not None:
#     print("\n--- ✅ COPD1 Patient Confidence Score Results ---")
#     print("'Avg_Score' is the model's confidence in the 'COPD1' prediction (closer to 1.0 is more confident).")
#     print("Patients with LOW scores might be outliers or less severe cases.")
#     # Use to_string() to ensure the full table is printed without truncation
#     print(copd1_confidence_df.to_string())

# # 2. Analyze COPD0 patients to see their risk scores (low is good)
# # We sort descending to see the "healthy" patients the model thinks are MOST at risk.
# copd0_risk_df = run_patient_analysis(
#     trained_model=model,
#     labels_path=LABEL_PATH,
#     audio_dir=AUDIO_DIR,
#     target_diagnosis='COPD0',
#     sort_ascending=False
# )

# if copd0_risk_df is not None:
#     print("\n--- ⚠️ COPD0 Patient Risk Assessment Results ---")
#     print("'Avg_Score' is the predicted risk (similarity to COPD1). Higher scores indicate higher risk.")
#     print("High-scoring patients may warrant closer clinical observation.")
#     # Use to_string() to ensure the full table is printed without truncation
#     print(copd0_risk_df.to_string())

In [73]:
# --- Step 8: Analyze Patient Scores and Provide Assessments ---

def run_patient_analysis(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    """
    Analyzes all patients with a specific diagnosis, computes the average model
    prediction score for their audio files, and returns a sorted DataFrame.
    """
    print(f"\n--- 🩺 Starting Analysis for '{target_diagnosis}' Patients ---")
    
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    
    if not patient_ids:
        print(f"No patients with diagnosis '{target_diagnosis}' found.")
        return None
        
    print(f"Found {len(patient_ids)} patients with '{target_diagnosis}' diagnosis. Analyzing files...")
    
    patient_predictions = {}
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fname = f"{pid}_{side}{i}.wav"
                fpath = os.path.join(audio_dir, fname)
                
                if not os.path.exists(fpath):
                    continue

                try:
                    y_audio, sr = librosa.load(fpath, sr=None)
                    log_mel_spec = extract_log_mel_spectrogram(y_audio, sr)
                    
                    model_input = np.expand_dims(log_mel_spec, axis=(0, -1))
                    
                    probability = trained_model.predict(model_input, verbose=0)[0][0]
                    scores.append(probability)
                except Exception as e:
                    print(f"Warning: Could not process file {fname}: {e}")
        
        if scores:
            formatted_scores = [f'{s:.4f}' for s in scores]
            print(f"  -> Scores for Patient {pid}: {formatted_scores}")
            patient_predictions[pid] = scores

    results_list = []
    for pid, scores in patient_predictions.items():
        avg_score = np.mean(scores)
        results_list.append({
            'Patient ID': pid,
            'Avg_Score': avg_score,
            'Audio_Files_Found': len(scores)
        })

    if not results_list:
        print("Could not generate any predictions.")
        return None

    results_df = pd.DataFrame(results_list)
    results_df = results_df.sort_values(by='Avg_Score', ascending=sort_ascending)
    return results_df.reset_index(drop=True)

# --- Define Assessment Functions ---

def assess_copd1_patient(row):
    """Generates an assessment text for a patient known to have COPD1."""
    # A score >= 0.5 means the model correctly leans towards a COPD1 diagnosis.
    if row['Avg_Score'] >= 0.5:
        return "✅ Model Confident (Correctly identified as COPD)"
    else:
        return "⚠️ Model Lacks Confidence (Possible miss / False Negative)"

def assess_copd0_patient(row):
    """Generates an assessment text for a healthy (COPD0) patient."""
    # A score < 0.5 means the model correctly leans towards a non-COPD diagnosis.
    if row['Avg_Score'] < 0.5:
        return "✅ Correctly Identified as Low-Risk"
    else:
        return "⚠️ Flagged as High-Risk (Possible early signs / False Positive)"

# --- Run the analysis and apply assessments ---

# 1. Analyze COPD1 patients 
copd1_confidence_df = run_patient_analysis(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD1',
    sort_ascending=True  # Show least confident patients first
)

if copd1_confidence_df is not None:
    # Add the new assessment column using the dedicated function
    copd1_confidence_df['Assessment'] = copd1_confidence_df.apply(assess_copd1_patient, axis=1)
    
    print("\n--- ✅ COPD1 Patient Confidence Score Results ---")
    print("'Avg_Score' > 0.5 indicates the model correctly identified the patient.")
    print("Patients with low scores might be outliers or less severe cases.")
    print(copd1_confidence_df.to_string())

# 2. Analyze COPD0 patients
copd0_risk_df = run_patient_analysis(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD0',
    sort_ascending=False  # Show highest risk "healthy" patients first
)

if copd0_risk_df is not None:
    # Add the new assessment column using the dedicated function
    copd0_risk_df['Assessment'] = copd0_risk_df.apply(assess_copd0_patient, axis=1)

    print("\n--- ⚠️ COPD0 Patient Risk Assessment Results ---")
    print("'Avg_Score' > 0.5 indicates a healthy patient is flagged as being at risk.")
    print("High-scoring patients may warrant closer clinical observation.")
    print(copd0_risk_df.to_string())


--- 🩺 Starting Analysis for 'COPD1' Patients ---
Found 5 patients with 'COPD1' diagnosis. Analyzing files...
  -> Scores for Patient H017: ['0.9670', '0.5199', '0.9883', '1.0000', '0.7341', '0.5714', '0.9976', '0.9645', '0.9610', '1.0000', '0.8247', '0.8901']
  -> Scores for Patient H029: ['0.9301', '0.9799', '0.9640', '0.9634', '0.9470', '0.9888', '0.9961', '0.9450', '0.9877', '0.9600', '0.9487', '0.9831']
  -> Scores for Patient H039: ['0.5356', '0.2112', '0.9805', '1.0000', '0.9033', '1.0000', '0.5471', '0.5108', '0.9307', '0.9999', '0.4237', '0.5611']
  -> Scores for Patient H043: ['0.9977', '0.9984', '0.9732', '0.8961', '0.5108', '0.9999', '0.9910', '0.9741', '0.9882', '0.9525', '0.5108', '1.0000']
  -> Scores for Patient H045: ['0.4789', '0.2349', '0.0251', '0.2030', '0.0054', '0.2811', '0.5453', '0.5367', '0.0374', '0.8421', '0.0071', '0.0971']

--- ✅ COPD1 Patient Confidence Score Results ---
'Avg_Score' > 0.5 indicates the model correctly identified the patient.
Patients with

Showing all 12 recording's prediction value and then counting

In [75]:
# --- Step 8 (Alternative): Analyze Patient Scores by Majority Vote ---

def run_patient_analysis_by_vote(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    """
    Analyzes patients using a majority vote method. For each patient, it counts
    the number of audio files predicted as COPD1 vs COPD0 and determines the
    final prediction based on the majority.
    """
    print(f"\n--- 🗳️ Starting Analysis for '{target_diagnosis}' Patients (Majority Vote Method) ---")
    
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    
    if not patient_ids:
        print(f"No patients with diagnosis '{target_diagnosis}' found.")
        return None
        
    print(f"Found {len(patient_ids)} patients with '{target_diagnosis}' diagnosis. Counting votes for each file...")
    
    patient_scores_data = {}
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fname = f"{pid}_{side}{i}.wav"
                fpath = os.path.join(audio_dir, fname)
                
                if not os.path.exists(fpath): continue

                try:
                    y_audio, sr = librosa.load(fpath, sr=None)
                    log_mel_spec = extract_log_mel_spectrogram(y_audio, sr)
                    model_input = np.expand_dims(log_mel_spec, axis=(0, -1))
                    probability = trained_model.predict(model_input, verbose=0)[0][0]
                    scores.append(probability)
                except Exception as e:
                    print(f"Warning: Could not process file {fname}: {e}")
        
        if scores:
            formatted_scores = [f'{s:.4f}' for s in scores]
            print(f"  -> Raw Scores for Patient {pid}: {formatted_scores}")
            patient_scores_data[pid] = scores

    results_list = []
    for pid, scores in patient_scores_data.items():
        # --- COUNTING LOGIC ---
        copd1_votes = sum(1 for s in scores if s >= 0.5)
        total_files = len(scores)
        copd0_votes = total_files - copd1_votes
        
        # Determine the final prediction by majority
        if copd1_votes > copd0_votes:
            final_prediction = 'COPD1'
        else:
            final_prediction = 'COPD0' # Default to COPD0 in case of a tie
            
        results_list.append({
            'Patient ID': pid,
            'COPD1_Votes': copd1_votes,
            'COPD0_Votes': copd0_votes,
            'Total_Files': total_files,
            'Final_Prediction': final_prediction
        })

    if not results_list:
        print("Could not generate any predictions.")
        return None

    results_df = pd.DataFrame(results_list)
    # Sort by the number of COPD1 votes to find the most/least severe cases
    results_df = results_df.sort_values(by='COPD1_Votes', ascending=sort_ascending)
    return results_df.reset_index(drop=True)

def assess_prediction_vs_truth(row, true_label):
    """Generates an assessment comparing the majority vote prediction to the known truth."""
    if row['Final_Prediction'] == true_label:
        return f"✅ Correct (Predicted {row['Final_Prediction']})"
    else:
        return f"❌ Incorrect (Predicted {row['Final_Prediction']}, but was {true_label})"

# --- Run the analysis using the counting method ---

# 1. Analyze COPD1 patients
copd1_vote_df = run_patient_analysis_by_vote(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD1',
    sort_ascending=True  # Show patients with the FEWEST COPD1 votes first
)

if copd1_vote_df is not None:
    copd1_vote_df['Assessment'] = copd1_vote_df.apply(assess_prediction_vs_truth, true_label='COPD1', axis=1)
    
    print("\n--- ✅ COPD1 Patient Majority Vote Results ---")
    print("This table shows if the model's majority vote matched the patient's actual 'COPD1' diagnosis.")
    print(copd1_vote_df.to_string())

# 2. Analyze COPD0 patients
copd0_vote_df = run_patient_analysis_by_vote(
    trained_model=model,
    labels_path=LABEL_PATH,
    audio_dir=AUDIO_DIR,
    target_diagnosis='COPD0',
    sort_ascending=False  # Show healthy patients with the MOST COPD1 votes first
)

if copd0_vote_df is not None:
    copd0_vote_df['Assessment'] = copd0_vote_df.apply(assess_prediction_vs_truth, true_label='COPD0', axis=1)

    print("\n--- ⚠️ COPD0 Patient Majority Vote Results ---")
    print("This table shows if the model incorrectly flagged a healthy patient based on a majority vote.")
    print("Incorrect '❌' assessments here are potential False Positives.")
    print(copd0_vote_df.to_string())


--- 🗳️ Starting Analysis for 'COPD1' Patients (Majority Vote Method) ---
Found 5 patients with 'COPD1' diagnosis. Counting votes for each file...
  -> Raw Scores for Patient H017: ['0.9670', '0.5199', '0.9883', '1.0000', '0.7341', '0.5714', '0.9976', '0.9645', '0.9610', '1.0000', '0.8247', '0.8901']
  -> Raw Scores for Patient H029: ['0.9301', '0.9799', '0.9640', '0.9634', '0.9470', '0.9888', '0.9961', '0.9450', '0.9877', '0.9600', '0.9487', '0.9831']
  -> Raw Scores for Patient H039: ['0.5356', '0.2112', '0.9805', '1.0000', '0.9033', '1.0000', '0.5471', '0.5108', '0.9307', '0.9999', '0.4237', '0.5611']
  -> Raw Scores for Patient H043: ['0.9977', '0.9984', '0.9732', '0.8961', '0.5108', '0.9999', '0.9910', '0.9741', '0.9882', '0.9525', '0.5108', '1.0000']
  -> Raw Scores for Patient H045: ['0.4789', '0.2349', '0.0251', '0.2030', '0.0054', '0.2811', '0.5453', '0.5367', '0.0374', '0.8421', '0.0071', '0.0971']

--- ✅ COPD1 Patient Majority Vote Results ---
This table shows if the model's